# Save the county details for every NOAA storm event into a Postgres table

## Install the Python requirements

In [ ]:
#%%bash
!pip install pandas==1.3.2
!pip install tqdm==4.62.2
!pip install psycopg2-binary==2.9.1
!pip install sqlalchemy==1.4.23

## Import the required packages

In [1]:
import numpy as np
import pandas as pd
import re
import io
import requests
import psycopg2

from tqdm.auto import tqdm  # for notebooks
from sqlalchemy import create_engine

from amazon_cred import ENDPOINT, PORT, USER, PASSWORD, DATABASE
from cred import CENSUS_API_KEY
from data_constants import NOAA_CSVFILES_URL, ZONE_COUNTY_CORR_URL, ZONE_COUNTY_CORR_CSV, TABLE_COLUMNS_NOAA
from data_constants import MIN_DAMAGE, BEGIN_YEAR, END_YEAR, STORM_CATEGORIES, BLS_CPI_CSV

In [5]:
df = pd.read_csv(BLS_CPI_CSV, index_col=False)
df = df.drop(columns=['Unnamed: 0'])
df.to_csv('bls')

,year,period,value
0,2019,12,256.974
1,2019,11,257.208
2,2019,10,257.346
3,2019,9,256.759
4,2019,8,256.558
...,...,...,...
255,2020,5,256.394
256,2020,4,256.389
257,2020,3,258.115
258,2020,2,258.678


## Constants

- North American Industry Classification System Code
- Regex pattern to remove square brackets from the response from census api

In [ ]:
TABLE_COLUMNS_CENSUS = ['ESTAB', 'PAYANN', 'EMP', 'NESTAB', 'NRCPOPT', 'POP']

NAICS_CODE_DICT = { # North American Industry Classification System Code
    '11': 'Agriculture, Forestry, Fishing, Hunting',
    '21': 'Mining',
    '22': 'Utilities',
    '23': 'Construction',
    '31-33': 'Manufacturing',
    '42': 'Wholesale',
    '44-45': 'Retail',
    '48-49': 'Transportation and Warehousing',
    '51': 'Information',
    '52': 'Finance and Insurance',
    '53': 'Real Estate Rental and Leasing',
    '54': 'Professional, Scientific, and Technical Services',
    '55': 'Management of Companies and Enterprises',
    '56': 'Administrative and Support and Waste Management and Remediation Services',
    '61': 'Educational Services',
    '62': 'Health Care and Social Assistance',
    '71': 'Arts, Entertainment, and Recreation',
    '72': 'Accommodation and Food Services',
    '81': 'Other Services (except Public Administration)',
    '92': 'Public Administration'
}

PATTERN = '\[\[|\[|\],|\]\]'
PATTERN = re.compile(PATTERN)

### Helper functions

In [4]:
def get_list_csvfiles(url):
    html = pd.read_html(url) # read_html and then get the list of files from the html table
    df = html[0]
    df.drop(columns=['Description'], inplace=True)
    df.dropna(inplace=True)
    df = df[df['Name'].str.contains('StormEvents_details.*d20', regex=True)]

    files_dict = {}
    for fname in df['Name']:
        result = re.findall('_d(?P<year>\d{4})', fname) # parse the year from the file name
        if len(result) > 0:
            files_dict[result[0]] = fname # dict key is the year, value is the filename
    return files_dict

def download_zone_county_corr_file_orig():
    df = pd.read_csv(ZONE_COUNTY_CORR_URL, sep='|', header=None, # Pipe delimited text
                names=['STATE', 'ZONE', 'CWA', 'NAME', 'STATE_ZONE',
                       'CZ_NAME', 'FIPS', 'TIME_ZONE', 'FE_AREA', 'LAT', 'LON'])
    df['STATE_FIPS'] = df['FIPS']//1000
    df = df[['STATE_FIPS', 'ZONE', 'FIPS']]
    df.to_csv('../data/zone_county_corr.csv', index=False)
    return df

def download_zone_county_corr_file(url):
    df = pd.read_csv(url)
    return df

def get_zone_county_corr_df():
    return download_zone_county_corr_file(ZONE_COUNTY_CORR_CSV)

## Retrieve storm data from NOAA

Data preparation steps:

1. Drop rows that have STATE_FIPS = NaN or CZ_FIPS = NaN
2. Convert DAMAGE_PROPERTY, DAMAGE_CROPS to numbers (replace Nan with 0)
3. Select only counties in 50 US states and Washington DC (FIPS ID for Wyoming is 56)
4. Select only records where DAMAGE_PROPERTY + DAMAGE_CROPS >= min_damage
5. Update the county FIPS for each record
6. Ensure FIPS are 5 characters (for Plotly)
7. Calculate the total damage in each county and build event list (some counties have more than one event)
8. Prepare the dataframe for visualization using Plotly choropleth maps

In [ ]:
def get_storm_data(year, files_dict):
    url = NOAA_CSVFILES_URL + files_dict[f'{year}']
    df_counties = pd.read_csv(url)
    zone_county_df = get_zone_county_corr_df()

    # 1. Drop rows that have STATE_FIPS = Nan or CZ_FIPS = Nan
    df_counties.dropna(subset=['STATE_FIPS', 'CZ_FIPS'], inplace=True)

    # 2. Convert DAMAGE_PROPERTY, DAMAGE_CROPS to numbers (replace Nan with 0)
    def convertStrToNum(st) -> float:
        '''
        convert str to float
        example: 3.00K --> 3000.0
        '''
        if type(st) is float:
            return 0.0 if np.isnan(st) else float(st)
        
        st = st.strip()
        if len(st) == 0:
            return 0.0
        elif st[-1] == 'k' or st[-1] == 'K':
            return float(st[:-1])*1_000.0 if len(st) > 1 else 1_000
        elif st[-1] == 'm' or st[-1] == 'M':
            return float(st[:-1])*1_000_000.0 if len(st) > 1 else 1_000_000
        elif st[-1] == 'b' or st[-1] == 'B':
            return float(st[:-1])*1_000_000_000.0 if len(st) > 1 else 1_000_000_000.0
        else:
            return 0.0

    df_counties['DAMAGE_PROPERTY'] = df_counties['DAMAGE_PROPERTY'].map(convertStrToNum)
    df_counties['DAMAGE_CROPS'] = df_counties['DAMAGE_CROPS'].map(convertStrToNum)

    # print(df_counties['STATE_FIPS'].unique()) # Puerto Rico is 99, American Samoa, US Virgin Islands

    # 3. Select only counties in 50 US states and Washington DC (FIPS ID for Wyoming is 56)
    # 4. DAMAGE_PROPERTY + DAMAGE_CROPS >= min_damage
    df_counties = df_counties[(df_counties['STATE_FIPS'] <= 56) & # for now only 50 US states
                          (df_counties['DAMAGE_PROPERTY'] + df_counties['DAMAGE_CROPS'] >= MIN_DAMAGE)] # either DAMAGE_PROPERTY or DAMAGE_CROPS > min_damage

    print(f"Number of counties with damage more or equal to than {MIN_DAMAGE} in {year}: {df_counties.shape[0]}")

    # 5. Update the county FIPS for each record
    def update_fips(row):
        if row['CZ_TYPE'] == 'C': # when CZ_TYPE is 'C', the record covers a single county
            row['FIPS'] = [int(row['STATE_FIPS'])*1000 + row['CZ_FIPS']] # calculate FIPS
            return row
        
        # CZ_TYPE is 'Z' (zone), get the county FIPS for each county that is in the zone
        st_id = row['STATE_FIPS']
        cz_id = row['CZ_FIPS']
        filter_rec = zone_county_df[(zone_county_df['STATE_FIPS'] == st_id) &
                                    (zone_county_df['ZONE'] == cz_id)]
        # more than 1 county can cover a zone
        lst_fips = [fips for fips in filter_rec['FIPS']]
        num_counties = len(lst_fips)
        if num_counties:
            row['FIPS'] = lst_fips
            row['DAMAGE_PROPERTY'] = row['DAMAGE_PROPERTY']/num_counties # split the damage across counties evenly!
            row['DAMAGE_CROPS'] = row['DAMAGE_CROPS']/num_counties
        else:
            row['FIPS'] = np.NaN
        return row

    df_counties = df_counties.apply(update_fips, axis=1)
    df_counties.dropna(subset=['FIPS'], inplace=True)
    df_counties = df_counties[df_counties['DAMAGE_PROPERTY'] + df_counties['DAMAGE_CROPS'] >= MIN_DAMAGE]
    df_counties = df_counties.explode(column=['FIPS'], ignore_index=True)

    # 6. Ensure FIPS are 5 characters (for plotly)
    df_counties['FIPS'] = df_counties['FIPS'].astype(str) # cast it as string in order to zfill
    df_counties['FIPS'] = df_counties['FIPS'].apply(lambda x: x.zfill(5))

    # 7. calculate the total damage in each county and build event list
    county_dict = {} # key is FIPS, value is a list [name, dict {EVENT_TYPE: BEGIN_YEARMONTH%100/BEGIN_DAY}, total damage]
    df_counties['TOTAL_DAMAGE'] = 0
    
    df_counties['BEGIN_TIME'] = df_counties['BEGIN_TIME'].astype(str)
    df_counties['BEGIN_TIME'] = df_counties['BEGIN_TIME'].str.zfill(4)
    df_counties['END_TIME'] = df_counties['END_TIME'].astype(str)
    df_counties['END_TIME'] = df_counties['END_TIME'].str.zfill(4)

    def calculate_total_damage(row):
        categories = STORM_CATEGORIES.keys()
        for k, v in STORM_CATEGORIES.items():
            searchStr = '|'.join(v)
            row[k] = re.search(searchStr, row['EVENT_TYPE']) != None

        county_id = row['FIPS']
        if county_id not in county_dict:
            county_dict[county_id] = [row['CZ_NAME'] + ', ' + row['STATE'], {}, 0, {}, 0, {}, 0, {}, 0]
            county_dict[county_id].extend([False]*len(categories))
        
        eventDateStr = str(row['BEGIN_YEARMONTH']%100) + '/' + str(row['BEGIN_DAY'])
        if row['EVENT_TYPE'] not in county_dict[county_id][1]:
            county_dict[county_id][1][row['EVENT_TYPE']] = set()
            county_dict[county_id][1][row['EVENT_TYPE']].add(eventDateStr)
        else:
            county_dict[county_id][1][row['EVENT_TYPE']].add(eventDateStr)

        county_dict[county_id][2] = county_dict[county_id][2] + row['DAMAGE_PROPERTY'] + row['DAMAGE_CROPS']
        for i, c in enumerate(categories):
            if row[c]:
                county_dict[county_id][i+9] = row[c]
                if row['EVENT_TYPE'] not in county_dict[county_id][2*i+3]:
                    county_dict[county_id][2*i+3][row['EVENT_TYPE']] = set()
                    county_dict[county_id][2*i+3][row['EVENT_TYPE']].add(eventDateStr)
                else:
                    county_dict[county_id][2*i+3][row['EVENT_TYPE']].add(eventDateStr)
                county_dict[county_id][2*i+4] += row['DAMAGE_PROPERTY'] + row['DAMAGE_CROPS']

        row['EVENT_DATE'] = pd.to_datetime(str(row['BEGIN_YEARMONTH']) + str(row['BEGIN_DAY']), format='%Y%m%d')
        row['NAME'] = row['CZ_NAME'] + ', ' + row['STATE']
        row['TOTAL_DAMAGE'] = row['DAMAGE_PROPERTY'] + row['DAMAGE_CROPS']
        
        begin_ts = pd.to_datetime(str(row['BEGIN_YEARMONTH'])[:4] + '-' + \
                                  str(row['BEGIN_YEARMONTH'])[4:] + '-' + \
                                  str(row['BEGIN_DAY']) + ':' + row['BEGIN_TIME'][:2] + ':' + row['BEGIN_TIME'][2:],
                                  format = '%Y-%m-%d:%H:%M')
        end_ts = pd.to_datetime(str(row['END_YEARMONTH'])[:4] + '-' + \
                                str(row['END_YEARMONTH'])[4:] + '-' + \
                                str(row['END_DAY']) + ':' + row['END_TIME'][:2] + ':' + row['END_TIME'][2:],
                                format = '%Y-%m-%d:%H:%M')
        row['DURATION'] = end_ts - begin_ts
        row['TORNADO_STRENGTH'] = ''
        if row['EVENT_TYPE'] == 'Tornado':
            row['TORNADO_STRENGTH'] = row['TOR_F_SCALE'] + ',' + \
                                      str(row['TOR_LENGTH']) + ',' + str(row['TOR_WIDTH'])
        
        return row

    df_counties = df_counties.apply(calculate_total_damage, axis=1)

    lst_columns = TABLE_COLUMNS_NOAA.copy() # make sure to take a copy of the list of columns
    lst_columns.extend(STORM_CATEGORIES.keys())
    df_counties = df_counties[lst_columns] # completes df_counties

    # prepare df_map for Plotly maps
    lst_columns = ['NAME', 'ALL_EVENTS', 'TOTAL_DAMAGE', 
                           'EVENT_TYPE_0', 'TYPE_0_DAMAGE',
                           'EVENT_TYPE_1', 'TYPE_1_DAMAGE',
                           'EVENT_TYPE_2', 'TYPE_2_DAMAGE']
    lst_columns.extend(STORM_CATEGORIES.keys())
    
    df_map = pd.DataFrame.from_dict(county_dict, orient='index', columns=lst_columns)
    df_map.reset_index(inplace=True)
    df_map.rename(columns={'index': 'FIPS'}, inplace=True)

    def build_event_type(row):
        event_desc = ''
        for k, v in row.items():
            if len(event_desc) != 0:
                event_desc += '; '
            event_desc += k + ': ' + ', '.join(v) # v is a set
        return event_desc
    df_map['ALL_EVENTS'] = df_map['ALL_EVENTS'].apply(build_event_type)
    df_map['EVENT_TYPE_0'] = df_map['EVENT_TYPE_0'].apply(build_event_type)
    df_map['EVENT_TYPE_1'] = df_map['EVENT_TYPE_1'].apply(build_event_type)
    df_map['EVENT_TYPE_2'] = df_map['EVENT_TYPE_2'].apply(build_event_type)

    return df_map, df_counties

## Get the population in a county using census.gov api

- American Community Survey 5-Year Data (2009-2019)

Ref: https://www.census.gov/data/developers/data-sets/acs-5year.html

Unlike the 1-year estimates, geographies do not have to meet a particular population threshold in order to be published.

- Decennial Census (2010, 2000)

Ref: https://www.census.gov/data/developers/data-sets/decennial-census.2000.html

In [ ]:
def get_results_from_census_api(url):
    r = requests.get(url)
    content = r.content # content is a Python string, parse the string to format the content into csv
    content = re.sub(PATTERN, "", content.decode('utf-8'))

    if len(r.content):
        content = r.content
        content = re.sub(PATTERN, "", content.decode('utf-8'))
        df = pd.read_csv(io.StringIO(content), quotechar='"')
        return df

    return None
    
def get_county_pop_census(year, county, state): # get population from decennial census in 2000, 2010
    url = f'https://api.census.gov/data/2000/dec/sf1?get=NAME,P001001&for=county:{county}&in=state:{state}&key={CENSUS_API_KEY}'
    df = get_results_from_census_api(url)
    if df is not None:
        url = f'https://api.census.gov/data/2010/dec/sf1?get=NAME,P001001&for=county:{county}&in=state:{state}&key={CENSUS_API_KEY}'
        df_2010 = get_results_from_census_api(url)
        if df_2010 is not None:
            pop_2000 = df.iloc[0]['P001001']
            pop_2010 = df_2010.iloc[0]['P001001']
            #print(county, ', ', state, ', ', pop_2000, ' to ', pop_2010)
            #print(pop_2000 + (year - 2000)*((pop_2010 - pop_2000)//10))
            # interpolate the population for years between 2001 and 2008 from population from year 2000 and 2010
            return pop_2000 + (year - 2000)*((pop_2010 - pop_2000)//10)
    return None            
        
def get_county_pop(df, year):
    if (year > 2019):
        year = 2019 # last year for which we have data available
    
    def get_pop(row):
        state = row.FIPS[:2]  # state FIPS is the first 2 characters of FIPS
        county = row.FIPS[2:] # county FIPS is the remaining characters of the 5 character FIPS
        # row['Features'] = ['Population']
        if year < 2009: # Population estimates program is from 2009 - 2019 only
            pop = get_county_pop_census(year, county, state)
            if pop is not None:
                row['DATA_COL'] = [str(pop)]
            else:
                print(f'No census pop record for {row["FIPS"]}')
                row['DATA_COL'] = ['NaN']
            return row
        
        # go to the ACS url to get population estimate
        url = f'https://api.census.gov/data/{year}/acs/acs5?get=NAME,B01001_001E&for=county:{county}&in=state:{state}&key={CENSUS_API_KEY}'
        df = get_results_from_census_api(url)
        if df is not None:
            row['DATA_COL'] = [str(df.iloc[0]['B01001_001E'])]
        else:
            print(f'No census pop record for {row["FIPS"]}')
            row['DATA_COL'] = ['NaN']
        return row

    df = df.progress_apply(get_pop, axis = 1) # use tqdm to monitor progress
    return df

## Get the county business patterns data

- County Business Patterns (1986-2019)

Ref: https://www.census.gov/data/developers/data-sets/cbp-nonemp-zbp.html

County Business Patterns provides annual statistics for businesses with paid employees within the U.S., Puerto Rico, and Island Areas at a detailed geography and industry level.

In [ ]:
def get_county_business_patterns(df, year):
    if (year > 2019):
        year = 2019 # last year for which we have data available
    def get_cbp(row):
        state = row.FIPS[:2]
        county = row.FIPS[2:]
        # row['Features'].extend(['County Business Patterns', '# establishments','Annual payroll($1000)','# employees'])
        row['DATA_COL'].extend([' ']) # empty string to separate the heading: 'County Business Patterns'

        if year >= 2015:
            url = f'https://api.census.gov/data/{year}/cbp?get=NAME,ESTAB,PAYANN,EMP&for=county:{county}&in=state:{state}&key={CENSUS_API_KEY}'
        else:
            url = f'https://api.census.gov/data/{year}/cbp?get=ESTAB,PAYANN,EMP&for=county:{county}&in=state:{state}&key={CENSUS_API_KEY}'
        r = requests.get(url)
        if len(r.content):
            content = r.content
            content = re.sub(PATTERN, "", content.decode('utf-8'))
            df = pd.read_csv(io.StringIO(content), quotechar='"')            
            row['DATA_COL'].extend([str(df.iloc[0]['ESTAB']), str(df.iloc[0]['PAYANN']), str(df.iloc[0]['EMP'])])
        else:
            print(f'No cbp record for {row["FIPS"]}')
            row['DATA_COL'].extend(['NaN']*3)
        return row

    df = df.progress_apply(get_cbp, axis=1)
    return df

## Get the non-employer statistics

- Nonemployer Statistics (1997-2018)

Ref: https://www.census.gov/data/developers/data-sets/cbp-nonemp-zbp.html

Nonemployer Statistics provides annual statistics on U.S. businesses with no paid employees or payroll at a detailed geography and industry level.

In [ ]:
def get_county_non_emp_stats(df, year): # upto 2018
    if (year > 2018):
        year = 2018
    def get_neb(row):
        state = row.FIPS[:2]
        county = row.FIPS[2:]
        # row['Features'].extend(['Nonemployer Statistics', '# establishments','Revenue($1,000)'])
        row['DATA_COL'].extend(' ') # empty string to separate the heading: 'Nonemployer Statistics'

        if year >= 2015:
            url = f'https://api.census.gov/data/{year}/nonemp?get=NAME,NESTAB,NRCPTOT&for=county:{county}&in=state:{state}&key={CENSUS_API_KEY}'
        else:
            url = f'https://api.census.gov/data/{year}/nonemp?get=NESTAB,NRCPTOT&for=county:{county}&in=state:{state}&key={CENSUS_API_KEY}'
        r = requests.get(url)
        if len(r.content):
            content = r.content
            content = re.sub(PATTERN, "", content.decode('utf-8'))
            df = pd.read_csv(io.StringIO(content), quotechar='"')
            
            row['DATA_COL'].extend([str(df.iloc[0]['NESTAB']), str(df.iloc[0]['NRCPTOT'])])
        else:
            print(f'No neb record for {row["FIPS"]}')
            row['DATA_COL'].extend(['NaN']*2)
        return row

    df = df.progress_apply(get_neb, axis=1)
    return df

## Get the county economic data

- Economic Census (2017, 2012, 2007, 2002)

Ref: https://www.census.gov/data/developers/data-sets/economic-census.html

The Economic Census is the official measure of the Nation’s businesses and economy. Conducted every five years, the survey serves as the statistical benchmark for current economic activity, such as, the Gross Domestic Product and the Producer Price Index.

In [ ]:
def get_county_economy(df, year):
    code_list = ''
    census_api_key = f'&key={CENSUS_API_KEY}'
    if year >= 2017:
        year = 2017
        base_url = f'https://api.census.gov/data/2017/ecnbasic?get=NAICS2017,ESTAB,EMP,RCPTOT&for=county:'#{county_fips}&in=state:{state_fips}'

        for code in NAICS_CODE_DICT.keys():
            code_list += ('&NAICS2017=' + code)
    elif year >= 2012:
        year = 2012
        base_url = f'https://api.census.gov/data/2012/ewks?get=NAICS2012,ESTAB,EMP,RCPTOT,OPTAX&for=county:'#{county_fips}&in=state:{state_fips}'
        for code in NAICS_CODE_DICT.keys():
            code_list += ('&NAICS2012=' + code)
    elif year >= 2007:
        year = 2007
        base_url = f'https://api.census.gov/data/2007/ewks?get=NAICS2007,ESTAB,EMP,RCPTOT,OPTAX&for=county:'#{county_fips}&in=state:{state_fips}'
        for code in NAICS_CODE_DICT.keys():
            code_list += ('&NAICS2007=' + code)
    else:
        year = 2002
        base_url = f'https://api.census.gov/data/2002/ewks?get=NAICS2002,ESTAB,EMP,RCPTOT,OPTAX&for=county:'#{county_fips}&in=state:{state_fips}'
        for code in NAICS_CODE_DICT.keys():
            code_list += ('&NAICS2002=' + code)

    def get_eco_data(row):
        # print(row.FIPS)
        state = row.FIPS[:2]
        county = row.FIPS[2:]
        # row['Features'].extend(['Economic Data', 'Rank #1 Industry', 'Value of business($1000)', '# establishments', '# employees'])
        # row['Features'].extend(['Rank #2 Industry', 'Value of business($1000)', '# establishments', '# employees'])
        # row['Features'].extend(['Rank #3 Industry', 'Value of business($1000)', '# establishments', '# employees'])
        row['DATA_COL'].extend(' ') # Empty string to separate the heading: Economic Data
            
        url = base_url + f'{county}&in=state:{state}'
        r = requests.get(url + code_list + census_api_key)
        if len(r.content):
            content = r.content

            content = re.sub(PATTERN, "", content.decode('utf-8'))
            code_col = f'NAICS{year}'
            df_eco = pd.read_csv(io.StringIO(content), quotechar='"')

            lst_cols = [code_col,'ESTAB','EMP','RCPTOT']
            df_eco = df_eco[lst_cols]

            df_eco = df_eco.groupby([code_col], as_index=False).sum()
            df_eco.sort_values('RCPTOT', ascending=False, inplace=True)

            for i in range(3):
                if df_eco.shape[0] > i:
                    row['DATA_COL'].extend([NAICS_CODE_DICT[str(df_eco.iloc[i][code_col])], 
                                          str(df_eco.iloc[i]['RCPTOT']), 
                                          str(df_eco.iloc[i]['ESTAB']), 
                                          str(df_eco.iloc[i]['EMP'])])
                else:
                    row['DATA_COL'].extend(['NaN']*4)
        else:
            print(f'No econ record for {row["FIPS"]}')
            row['DATA_COL'].extend(['NaN']*12)
        return row
    
    df = df.progress_apply(get_eco_data, axis=1)
    return df

## Main loop for getting the above data for every county in the storms data for a given year

- Use tqdm to monitor progress
- Save the data collected into the Amazon RDS database

In [ ]:
files_dict = get_list_csvfiles(NOAA_CSVFILES_URL)

# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

df_db = None
for year in range(BEGIN_YEAR, END_YEAR):
    df_map, df_counties = get_storm_data(year, files_dict)

    df_counties = get_county_pop(df_counties, year)

    df_counties = get_county_business_patterns(df_counties, year)

    df_counties = get_county_non_emp_stats(df_counties, year)
    
    df_counties = get_county_economy(df_counties, year)
    
    df_counties['Year'] = year
    if df_db is not None:
        df_db = pd.concat([df_db, df_counties], axis=0, ignore_index=True)
    else:
        df_db = df_counties
        
df_db['DATA_COL'] = df_db['DATA_COL'].apply(lambda x : '|'.join(x))
df_db['DATA_COL'] = df_db['DATA_COL'].astype(str)
df_db

# save the dataframe into Amazon RDS
engine = create_engine(f'postgresql+psycopg2://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}')
with engine.begin() as connection:
    df_db.to_sql('counties', con=connection, if_exists='append',index=False)
engine.dispose()

## Save the database into a backup csv file

In [ ]:
conn = psycopg2.connect(
    host=ENDPOINT,
    port=PORT,
    user=USER,
    password=PASSWORD,
    database = DATABASE
)
with conn:
    select = """SELECT * from counties"""
    df_counties = pd.read_sql(select, con=conn)
    #delete = """DELETE FROM counties WHERE "Year" = 2004"""
    #conn.cursor().execute(delete)
conn.close()

# import copy
# df_safecopy = pd.DataFrame(columns=df_counties.columns, data=copy.deepcopy(df_counties.values))
# df_counties
# df_counties.to_csv('../data/df_counties.csv') # Save a backup copy of the database